In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.feature_column as fc 
import tensorflow.debugging as tfdbg
from sklearn.model_selection import train_test_split

df = pd.read_csv("data/train.flat.csv")
for col in df.columns:
    if df[col].fillna('').nunique() == 1:
        print("column skipped (cardinality = 1):", col)
        df = df.drop(col, axis=1)

In [ ]:
np.log(df['totals.transactionRevenue']).hist(bins=50)